# 🔎 Face Search: One-to-Many Face Matching

<div style="display:flex; flex-wrap:wrap; align-items:center;">
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pepy.tech/projects/uniface"><img alt="PyPI Downloads" src="https://static.pepy.tech/badge/uniface"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pypi.org/project/uniface/"><img alt="PyPI Version" src="https://img.shields.io/pypi/v/uniface.svg"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://opensource.org/licenses/MIT"><img alt="License" src="https://img.shields.io/badge/License-MIT-blue.svg"></a>
  <a style="margin-bottom:6px;" href="https://github.com/yakhyo/uniface"><img alt="GitHub Stars" src="https://img.shields.io/github/stars/yakhyo/uniface.svg?style=social"></a>
</div>

**UniFace** is a lightweight, production-ready, all-in-one face analysis library built on ONNX Runtime.

🔗 **GitHub**: [github.com/yakhyo/uniface](https://github.com/yakhyo/uniface) | 📚 **Docs**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)

---

## 📖 Overview

This notebook demonstrates **face search** - finding a person in a group of people:

- ✅ Build a face database from a group photo
- ✅ Search for a specific person using a query image
- ✅ Rank matches by similarity score

**Use cases**: Photo organization, security systems, social media tagging

## 1️⃣ Installation

In [ ]:
%pip install -q uniface

import os
import urllib.request

os.makedirs('assets', exist_ok=True)

BASE_URL = "https://raw.githubusercontent.com/yakhyo/uniface/main/assets"
images = ["einstien.png", "scientists.png"]

for img in images:
    if not os.path.exists(f'assets/{img}'):
        urllib.request.urlretrieve(f"{BASE_URL}/{img}", f"assets/{img}")
        print(f"✓ Downloaded {img}")

## 2️⃣ Import Libraries

In [ ]:
import cv2
import matplotlib.pyplot as plt

import uniface
from uniface import FaceAnalyzer
from uniface.detection import RetinaFace
from uniface.recognition import ArcFace

print(f"UniFace version: {uniface.__version__}")

## 3️⃣ Initialize Face Analyzer

In [ ]:
analyzer = FaceAnalyzer(
    detector=RetinaFace(confidence_threshold=0.5),
    recognizer=ArcFace()
)

## 4️⃣ Load Query Image (Who We're Looking For)

We'll use Einstein's photo as our query to find him in a group photo.

In [ ]:
einstein_path = 'assets/einstien.png'
einstein_image = cv2.imread(einstein_path)

einstein_faces = analyzer.analyze(einstein_image)

if einstein_faces:
    einstein_face = einstein_faces[0]
    print(f'✓ Query face loaded with {einstein_face.embedding.shape[1]}D embedding')

## 5️⃣ Load Group Photo (Database to Search)

In [ ]:
group_photo_path = 'assets/scientists.png'
group_photo = cv2.imread(group_photo_path)

group_faces = analyzer.analyze(group_photo)
print(f'✓ Found {len(group_faces)} people in the group photo')

## 6️⃣ Display Query and Search Database

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].imshow(cv2.cvtColor(einstein_image, cv2.COLOR_BGR2RGB))
axes[0].set_title("🔍 Query: Who we're looking for", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(group_photo, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'🗄️ Database: Group of {len(group_faces)} scientists', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 7️⃣ Search: Compare Query Against All Faces

In [ ]:
if not einstein_faces or not group_faces:
    print('⚠ Error: Could not detect faces')
else:
    matches = []
    for i, person in enumerate(group_faces):
        similarity = einstein_face.compute_similarity(person)
        matches.append((i, similarity))

    matches.sort(key=lambda x: x[1], reverse=True)

    print('📊 Top 5 Most Similar People:')
    print('-' * 40)
    for rank, (person_idx, similarity) in enumerate(matches[:5], 1):
        status = "✅" if similarity > 0.6 else "❌"
        print(f'{rank}. Person #{person_idx + 1}: {similarity:.4f} {status}')

## 8️⃣ Visualize Best Match

In [ ]:
if einstein_faces and group_faces:
    best_match_idx, best_similarity = matches[0]
    result_image = group_photo.copy()

    for i, person in enumerate(group_faces):
        bbox = person.bbox.astype(int)
        if i == best_match_idx:
            color = (0, 255, 0)
            thickness = 3
        else:
            color = (128, 128, 128)
            thickness = 1
        cv2.rectangle(result_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, thickness)
        if i == best_match_idx:
            label = f'Match: {best_similarity:.3f}'
            cv2.putText(result_image, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.title(f'🎯 Best Match: Person #{best_match_idx + 1} (similarity: {best_similarity:.3f})', fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

---

## 📝 Summary

| Feature | Description |
|---------|-------------|
| **Query** | Single face to search for |
| **Database** | Group of faces to search in |
| **Matching** | Compare query against all database faces |
| **Ranking** | Sort by similarity score (descending) |
| **Threshold** | 0.6 for positive match |

### Scaling Tips

- For large databases, consider using **vector databases** (FAISS, Milvus)
- Embeddings are 512D vectors suitable for ANN search
- Pre-compute embeddings for faster search

---

## 🔗 Next Steps

- **Face Analysis**: Age, gender prediction → [05_face_analyzer.ipynb](./05_face_analyzer.ipynb)
- **Face Parsing**: Semantic segmentation → [06_face_parsing.ipynb](./06_face_parsing.ipynb)
- **Full Documentation**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)